In [5]:
%load_ext autoreload
%autoreload 2
import tqdm

from pycoq.agent import evaluate_agent
from pycoq.agent import evaluate_agent_in_session
from pycoq.agent import auto_agent as agent

import pycoq.serapi

from typing import List, Iterable
import asyncio

import pymongo
db =  pymongo.MongoClient().formulas.propositional.implication


def evaluate(e, op, arg: List[bool]):
    '''
    Boolean stack evaluator
    e: is binary tree expression in stack form
    arg: is the list of boolean values
    op: (bool, bool) -> bool binary operator in the tree nodes
    '''
    stack = []
    pos = 0
    while pos < len(e):
        c = e[pos]; pos += 1  # read element from expression postfix
        if c > 0 and c <= len(arg): # normal argument, place on stack
            stack.append(arg[c-1])
        elif c == -2:    # binary operator
            if len(stack) >= 2:
                a = stack.pop()
                b = stack.pop()
                res = op(b, a)
                stack.append(res)
            else:
                raise ValueError("bad postfix: not enough args for binary operator")
        else:
            raise ValueError("bad postfix: unknown element code")
    if len(stack) == 1:
        return stack.pop()
    else:
        raise ValueError("bad postfix, too many args for binary operators")

        

def sexp_op(a, b):
    #arrow = "\u2192"
    arrow = "->"
    return f"({a}{arrow}{b})"

def imply(a: bool, b: bool):
    return (not a) or b

def sexp(e): 
    return evaluate(e, sexp_op, [chr(ord('A')+i) for i in range(max(e))])

def coq_proposition(postfix, n_args, n_vars, lex_pos) -> str:
    theorem_name = f"th_{n_args}_{n_vars}_{lex_pos}"
    prop_formula = sexp(postfix)
    prop_variables = " ".join([chr(ord('A')+i) for i in range(n_vars)])
    return f"Theorem {theorem_name}: forall {prop_variables}: Prop, {prop_formula}."

# mark by * where bool_denote judgement of proposition does not agree with type inhabitation judgement 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
cfg = pycoq.common.serapi_kernel_config()

n_args = 4
n_vars = 2

pycoq.agent.DEBUG = False
for e in db.find({'n_args':n_args, 'n_vars':n_vars}):
    th = coq_proposition(e['postfix'], n_args, n_vars, e['lex_pos'])
    res = await evaluate_agent(cfg, agent, th, agent_parameters = {'auto_limit': 10})
    print( '*' if e['bool_denote'] != (res > 0) else '', th, res, e['bool_denote'])

 Theorem th_4_2_0: forall A B: Prop, (A->(A->(A->B))). -1 False
 Theorem th_4_2_1: forall A B: Prop, (A->(A->(B->A))). 1 True
 Theorem th_4_2_2: forall A B: Prop, (A->(A->(B->B))). 1 True
 Theorem th_4_2_3: forall A B: Prop, (A->(B->(A->A))). 1 True
 Theorem th_4_2_4: forall A B: Prop, (A->(B->(A->B))). 1 True
 Theorem th_4_2_5: forall A B: Prop, (A->(B->(B->A))). 1 True
 Theorem th_4_2_6: forall A B: Prop, (A->(B->(B->B))). 1 True
 Theorem th_4_2_7: forall A B: Prop, (A->((A->A)->B)). -1 False
 Theorem th_4_2_8: forall A B: Prop, (A->((A->B)->A)). 1 True
 Theorem th_4_2_9: forall A B: Prop, (A->((A->B)->B)). 2 True
 Theorem th_4_2_10: forall A B: Prop, (A->((B->A)->A)). 1 True
 Theorem th_4_2_11: forall A B: Prop, (A->((B->A)->B)). -1 False
 Theorem th_4_2_12: forall A B: Prop, (A->((B->B)->A)). 1 True
 Theorem th_4_2_13: forall A B: Prop, (A->((B->B)->B)). -1 False
 Theorem th_4_2_14: forall A B: Prop, ((A->A)->(A->B)). -1 False
 Theorem th_4_2_15: forall A B: Prop, ((A->A)->(B->A)).

In [7]:
cfg = pycoq.common.serapi_kernel_config()

n_args = 4
n_vars = 2


pycoq.agent.DEBUG = False

async with pycoq.serapi.CoqSerapi(cfg, logfname="logtemp") as coq:
    for e in tqdm.tqdm(db.find({'n_args':n_args, 'n_vars':n_vars})):
        prop = coq_proposition(e['postfix'], n_args, n_vars, e['lex_pos'])
        res = await evaluate_agent_in_session(coq, agent, prop, agent_parameters = {'auto_limit': 10})
        print( '*' if e['bool_denote'] != (res > 0) else '', prop, res, e['bool_denote'])

22it [00:00,  5.16it/s]

 Theorem th_4_2_0: forall A B: Prop, (A->(A->(A->B))). -1 False
 Theorem th_4_2_1: forall A B: Prop, (A->(A->(B->A))). 1 True
 Theorem th_4_2_2: forall A B: Prop, (A->(A->(B->B))). 1 True
 Theorem th_4_2_3: forall A B: Prop, (A->(B->(A->A))). 1 True
 Theorem th_4_2_4: forall A B: Prop, (A->(B->(A->B))). 1 True
 Theorem th_4_2_5: forall A B: Prop, (A->(B->(B->A))). 1 True
 Theorem th_4_2_6: forall A B: Prop, (A->(B->(B->B))). 1 True
 Theorem th_4_2_7: forall A B: Prop, (A->((A->A)->B)). -1 False
 Theorem th_4_2_8: forall A B: Prop, (A->((A->B)->A)). 1 True
 Theorem th_4_2_9: forall A B: Prop, (A->((A->B)->B)). 2 True
 Theorem th_4_2_10: forall A B: Prop, (A->((B->A)->A)). 1 True
 Theorem th_4_2_11: forall A B: Prop, (A->((B->A)->B)). -1 False
 Theorem th_4_2_12: forall A B: Prop, (A->((B->B)->A)). 1 True
 Theorem th_4_2_13: forall A B: Prop, (A->((B->B)->B)). -1 False
 Theorem th_4_2_14: forall A B: Prop, ((A->A)->(A->B)). -1 False
 Theorem th_4_2_15: forall A B: Prop, ((A->A)->(B->A)).

35it [00:00, 66.96it/s]

 Theorem th_4_2_32: forall A B: Prop, (((A->B)->A)->B). -1 False
 Theorem th_4_2_33: forall A B: Prop, (((A->B)->B)->A). -1 False
 Theorem th_4_2_34: forall A B: Prop, (((A->B)->B)->B). -1 False


In [8]:
import tqdm
from serlib.parser import SExpParser
async with pycoq.serapi.CoqSerapi(cfg, logfname="logtemp") as coq:
    parser = SExpParser()

    for cnt in tqdm.tqdm(range(10000)):
        result = await coq.execute(f"Theorem th_{cnt}: forall A: Prop, A->A. auto. Qed. \n")

        if (len(result[2]) > 0):
            break

        last_sids = result[-1]



100%|██████████| 10000/10000 [00:15<00:00, 626.19it/s]


In [9]:
import json
with open("logtemp") as logfile:
    j = json.load(logfile)

j['response'][-10:]

['(Feedback((doc_id 0)(span_id 30000)(route 0)(contents Processed)))\n',
 '(Answer 39998 Completed)\n',
 '(Answer 39999 Ack)\n',
 '(Feedback((doc_id 0)(span_id 30001)(route 0)(contents(ProcessingIn master))))\n',
 '(Feedback((doc_id 0)(span_id 30000)(route 0)(contents Processed)))\n',
 '(Feedback((doc_id 0)(span_id 29999)(route 0)(contents Processed)))\n',
 '(Feedback((doc_id 0)(span_id 30001)(route 0)(contents Processed)))\n',
 '(Answer 39999 Completed)\n',
 '(Answer EOF Ack)\n',
 '(Answer EOF Completed)\n']

In [10]:
with open("test.in","w") as f:
    f.writelines(x + "\n" for x in j['sent'])

In [11]:
j['sent'][-10:]

['(Exec 29994)',
 '(Exec 29995)',
 '(Add () "Theorem th_9998: forall A: Prop, A->A. auto. Qed. \n")',
 '(Exec 29996)',
 '(Exec 29997)',
 '(Exec 29998)',
 '(Add () "Theorem th_9999: forall A: Prop, A->A. auto. Qed. \n")',
 '(Exec 29999)',
 '(Exec 30000)',
 '(Exec 30001)']

In [12]:
j['stderr'][-100:]

[]

In [52]:
with open('test130k.v','w') as f:
    for cnt in range(130000):
        f.write(f"Theorem th_{cnt}: forall A: Prop, A->A. auto. Qed. \n")


In [60]:
with open('test400000.v','w') as f:
    for cnt in range(400000):
        f.write(f"Theorem th_{cnt}: forall A: Prop, A->A. auto. Qed. \n")
